In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [ ]:
root_path = '/content/drive/My Drive/data'
for item in os.listdir(root_path):
    item_path = os.path.join(root_path, item)
    if os.path.isdir(item_path):
        print(f'Папка: {item}')
    else:
        print(f'Файл: {item}')

Папка: running
Папка: walking
Папка: idle
Папка: stairs


In [ ]:
activity_mapping = {
    'running': 'running',
    'walking': 'walking',
    'idle': 'idle',
    'stairs': 'stairs'
}

In [ ]:
def calculate_time_features(df):
    time_features = df.groupby('activities').agg(['mean', 'std', 'min', 'max'])
    time_features.columns = ['_'.join(col).strip() for col in time_features.columns.values]
    return time_features

In [ ]:
df_list = []
for folder_name in os.listdir(root_path):
    folder_path = os.path.join(root_path, folder_name)
    if os.path.isdir(folder_path):
        folder_df_list = []
        activity = activity_mapping[folder_name]
        for file_name in os.listdir(folder_path):
            if file_name.endswith('.csv'):
                file_path = os.path.join(folder_path, file_name)
                df = pd.read_csv(file_path)
                df = df.assign(activities=activity)
                folder_df_list.append(df)
        if folder_df_list:
            combined_folder_df = pd.concat(folder_df_list)
            df_list.append(combined_folder_df)

combined_df = pd.concat(df_list)
print(combined_df.head())

   accelerometer_X  accelerometer_Y  accelerometer_Z activities
0       -22.998701        31.598675       -26.297910    running
1        -5.942408         2.935291       -11.113884    running
2         7.886501        14.863204        -8.508993    running
3       -11.836933         1.216255        -8.456321    running
4       -19.306843       -13.919888         3.581726    running


In [97]:
combined_df['accelerometer_magnitude'] = (combined_df['accelerometer_X']**2 +
                                          combined_df['accelerometer_Y']**2 +
                                          combined_df['accelerometer_Z']**2)**0.5
combined_df['accelerometer_speed'] = combined_df['accelerometer_magnitude'].diff().fillna(0)
combined_df['accelerometer_jerk'] = combined_df['accelerometer_speed'].diff().fillna(0)

In [98]:
X = combined_df[['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z',
                 'accelerometer_magnitude', 'accelerometer_speed', 'accelerometer_jerk']]
y = combined_df['activities']

In [99]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [100]:
numeric_features = X.columns.tolist()
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features)])

In [101]:
svm_model = SVC()
rf_model = RandomForestClassifier()

In [102]:
svm_pipe = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', svm_model)])

rf_pipe = Pipeline(steps=[('preprocessor', preprocessor),
                          ('classifier', rf_model)])

In [103]:
svm_pipe.fit(X_train, y_train)
rf_pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['accelerometer_X',
                                                   'accelerometer_Y',
                                                   'accelerometer_Z',
                                                   'accelerometer_magnitude',
                                                   'accelerometer_speed',
                                                   'accelerometer_jerk'])])),
                ('classifier', RandomForestClassifier())])

In [104]:
svm_preds = svm_pipe.predict(X_test)
rf_preds = rf_pipe.predict(X_test)

In [106]:
svm_report = classification_report(y_test, svm_preds)
rf_report = classification_report(y_test, rf_preds)

print("Classification Report for SVM:")
print(svm_report)

print("Classification Report for Random Forest:")
print(rf_report)

Classification Report for SVM:
              precision    recall  f1-score   support

        idle       0.98      0.99      0.98      6221
     running       0.95      0.93      0.94     20510
      stairs       1.00      0.01      0.01       915
     walking       0.84      0.93      0.88     11126

    accuracy                           0.92     38772
   macro avg       0.94      0.72      0.71     38772
weighted avg       0.93      0.92      0.91     38772

Classification Report for Random Forest:
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00      6221
     running       1.00      1.00      1.00     20510
      stairs       1.00      0.97      0.99       915
     walking       0.99      1.00      1.00     11126

    accuracy                           1.00     38772
   macro avg       1.00      0.99      0.99     38772
weighted avg       1.00      1.00      1.00     38772

